<a href="https://colab.research.google.com/github/bdadeveloper1/MachineLearningProjects/blob/main/hazelFinancialChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Hazel the Financial QuizBot Project** 
## Data Science Team:
**Data Science Manager** - Anade Davis

**Lead Data Scientist** - Ragavendhra Ramanan 

**Lead Machine Learning Engineer** - Tanjeel Ahmed 

**Lead Data Science Researcher** - Hafizah Ab Rahim

**Data Scientist** - Ivy Hsu

**Data Engineer** - Brandon Oppong-Antwi

**Data Science Researcher** -  Michael Ayedun

**Data Science Researcher** - Alexander Lai 

### Resources

https://corporatefinanceinstitute.com/resources/knowledge/tests/finance-test/
https://www.funtrivia.com/playquiz/quiz32000124a2580.html
https://medium.com/voice-tech-podcast/how-to-build-a-chatbot-quiz-for-whatsapp-917b9c52bea
https://github.com/piyush-jaiswal/quizbot
https://medium.com/voice-tech-podcast/how-to-build-a-chatbot-quiz-for-whatsapp-917b9c52bea


In [ ]:
# Setup
!pip install -q wordcloud #is a visual representation of text data. 
!pip install nltk #Natural Language toolkit used to work with human language data
!pip install punkt #Pakcage within ntlk which divides a text into a listen of sentences to tokenize them

ERROR: Could not find a version that satisfies the requirement punkt (from versions: none)
ERROR: No matching distribution found for punkt


In [ ]:
import wordcloud #represent word data

import nltk #natural language processing with text recognition subscripts
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
import json #is a subset of JavaScript syntax 
from pprint import pprint #support to pretty-print lists, tuples, & dictionaries recursively.
import random #random variable generation. Used for random question picking
from random import randint #Return random integer in range [a, b], including both end points.
import os #operation system
import nltk #language processing
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords #language processing for stop word function commands
from nltk.stem import WordNetLemmatizer #Returns the input word unchanged if it cannot be found in WordNet.
import string

WUP_Threshold = 0.8
greeting = ['Hi there, how are you doing today!\nI am Hazel, the financial chatbot let\'s play a quiz today!, \n You can choose between a finance or a random brain teasers quiz.', 'Hi I\'m Hazel, your personal financial quiz master!', 'Hazel here, let\'s start']
wrong_category = ['Hmm! I\' m sorry friend We currently do not have any quizzes on that.', 'Sorry we don\'t have any questions regarding that.', 'Hmm, pick another one!']
next_question = ['Here comes the next question!', 'Get ready for the next one!', 'Here comes another']
correct_answer = ['Correct answer, well done!', 'You\'re smart! You must be a financial expert', 'You\'re a genius!']
wrong_answer = ['Sorry, wrong answer. Nevermind', 'Wrong! C\'mon you can do better', 'Nah! That is not correct!']


def extractNounsAndAdjectives(words):
        NJ = []
        pos = nltk.pos_tag(words)
        for word, tag in pos:
                if tag[0] in ('N', 'J'):
                        NJ.append(word)
        return NJ

# Wu-Palmer similarity
# Calculates relatedness by considering the depths of the two synsets in the WordNet taxonomies, along with the depth of the LCS (Least Common Subsumer).
def WUPSimilarity(w1, w2):
        w1 = wn.synsets(w1)
        w2 = wn.synsets(w2)
        max_WUP = 0
        # Checking for the first 3 synonyms in order to avoid noise
        for i in range(0, min(3, len(w1))):
                for j in range(0, min(3, len(w2))):
                        sim = w1[i].wup_similarity(w2[j])
                        if sim is not None:
                                max_WUP = max(max_WUP, sim)
        return max_WUP

# Compute possible categories based on user response
def computeCategories(categories, user_response_NJ):
        probable_categories = {}
        for w1 in categories:
                for w2 in user_response_NJ:
                        for w in w1.split('-'):
                                sim = WUPSimilarity(w, w2)
                                # print(w, w2, sim)
                                if sim >= 1:
                                        probable_categories.clear()
                                        probable_categories[w1] = w2
                                        return probable_categories
                                elif sim >= WUP_Threshold:
                                        probable_categories[w1] = w2
        return probable_categories

# Compute the possible choices present in user_response
def computeChoices(user_response, choices):
        # # Confirm for ambiguous user answers
        
        punctuations = list(string.punctuation)
        stop_words = set(stopwords.words('english'))
        lemmatizer = WordNetLemmatizer()
        ct = 'a'
        probable_choices = []
        user_response = nltk.word_tokenize(user_response.lower())
        user_response = [lemmatizer.lemmatize(word, pos='v') for word in user_response]
        user_response = [i for i in user_response if i not in punctuations and i not in stop_words]
        user_response = set(user_response)
        for choice in choices:
                # print(choice)
                choice = nltk.word_tokenize(choice.lower())
                choice = [lemmatizer.lemmatize(word, pos='v') for word in choice]
                choice = [i for i in choice if i not in punctuations and i not in stop_words]
                choice = set(choice)
                if len(user_response.intersection(choice)) > 0:
                        probable_choices.append(ct)
                ct = chr(ord(ct) + 1)
        return probable_choices

def displayBotResponse(score, responses, isCorrect, answer_choice=''):
        print(random.choice(responses))
        if isCorrect:
                score += 1
        else:
                score -= 0.25
                print("The correct answer is {}".format(answer_choice.upper()))
        print ('Score = {}'.format(score))
        return score

def loadAllCategories():
        categories = set()
        for root, dirs, files in os.walk("/content/triviaJSONFiles"):
                for name in files:
                        categories.add(name[: -5])
        return categories

def chooseCategory():
        categories = loadAllCategories()
        flag = False
        category = ""
        print('What would you liked to be quizzed on?')
        while flag == False:
                user_response = input()
                print()
                if len(user_response) > 0 and user_response[0] == '@':
                        if user_response[1: ] == 'list_quizzes':
                                for c in categories:
                                        print(c)
                                print()
                                continue
                words = nltk.word_tokenize(user_response)
                user_response_NJ = extractNounsAndAdjectives(words)
                probable_categories = computeCategories(categories, user_response_NJ)

                if len(probable_categories) == 0:
                        print("{} Type @list_quizzes to list categories".format(random.choice(wrong_category)))
                elif len(probable_categories) == 1:
                        category = list(probable_categories.keys())[0]
                        print('Alright! One quiz on {}({}) coming right up!'.format(category, probable_categories[category]))
                        print('We score +1 for a correct answer and -0.25 for a wrong one. Enjoy!')
                        print('Type @stop_quiz anytime to quit the quiz')
                        print('To change topics type @change_quiz\n')
                        flag = True
                else:
                        print("OK, judging by your response I have multiple categories.")
                        keys = probable_categories.keys()
                        for key in keys:
                                print("{}({})".format(key, probable_categories[key]))
                        print()
                        print("Which one would you like?")
                        print("To list all my categories, you can type @list_quizzes\n")
        return category

def loadCategoryData(category):
        # Open quiz bank for particular category
        dir_path = os.path.join('/content/triviaJSONFiles')
        with open(os.path.join(dir_path, category + '.json')) as data_file:
                data = json.load(data_file)
        return data

def displayScores(category_scores, total_score):
        keys = category_scores.keys()
        print('Here are your categorical scores')
        for key in keys:
                print("{}: {}".format(key, category_scores[key]))
        print('Your total score is {}'.format(total_score))


def quiz():
        print(random.choice(greeting))
        category = chooseCategory()
        data = loadCategoryData(category)

        # Start quiz!
        genarated = []
        stop = False
        score = 0
        category_scores = {}
        total_score = 0

        print('\nHere we go!')
        while stop == False:
                rand = randint(-1, len(data) - 1)
                while rand in genarated:
                        rand = randint(-1, len(data) - 1)
                genarated.append(rand)

                print(data[rand]['question'])
                ct = 'A'
                answer = data[rand]['answer']
                answer_choice = ''
                for c in data[rand]['choices']:
                        print('{}. {}'.format(ct, c))
                        if c == answer:
                                answer_choice = ct
                        ct = chr(ord(ct) + 1)

                unambiguous_response = False
                while unambiguous_response == False:
                        user_response = input()
                        print()
                        if len(user_response) > 0 and user_response[0] == '@':
                                if user_response[1: ] == 'stop_quiz':
                                        unambiguous_response = True
                                        stop = True
                                        category_scores[category] = score
                                        total_score += score
                                        displayScores(category_scores, total_score)
                                        print('Thanks for playing. Until next time!')
                                        continue
                                elif user_response[1: ] == 'change_quiz':
                                        unambiguous_response = True
                                        category_scores[category] = score
                                        total_score += score
                                        category = chooseCategory()
                                        data = loadCategoryData(category)
                                        displayScores(category_scores, total_score)
                                        continue
                        response_words = nltk.word_tokenize(user_response.lower())
                        answer_choice = answer_choice.lower()
                        probable_choices = computeChoices(user_response, data[rand]['choices'])
                        # print(probable_choices)

                        if answer_choice in response_words:
                                score = displayBotResponse(score, correct_answer, True)
                                unambiguous_response = True
                        elif len(probable_choices) >= 1:
                                if len(probable_choices) == 1:
                                        unambiguous_response = True
                                        if probable_choices[0].lower() == answer_choice:
                                                score = displayBotResponse(score, correct_answer, True)
                                        else:
                                                score = displayBotResponse(score, wrong_answer, False, answer_choice)
                                else:
                                        print('Hmm. You seem to have picked more than one option. Pick one')
                                        print(probable_choices)
                        else:
                                score = displayBotResponse(score, wrong_answer, False, answer_choice)
                                unambiguous_response = True
                if stop == False:
                        print(random.choice(next_question))
                print()


if __name__ == '__main__':
        quiz()


Hi there, how are you doing today!
I am Hazel, the financial chatbot let's play a quiz today!, 
 You can choose between a finance or a random brain teasers quiz.
What would you liked to be quizzed on?
finance

Alright! One quiz on finance(finance) coming right up!
We score +1 for a correct answer and -0.25 for a wrong one. Enjoy!
Type @stop_quiz anytime to quit the quiz
To change topics type @change_quiz


Here we go!
To ensure that some of your retirement savings will not be subject to income tax upon withdrawal, you would contribute to:
A. A traditional individual retirement account (IRA)
B. A Roth IRA
C. A 401(k) plan


KeyboardInterrupt: ignored